### Imports

In [23]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F # for relu, tanh, sigmoid
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Create Fully Connected Network

In [2]:
class NN(nn.Module):

    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


### Testing Code

- 64 random images with image size (28x28) = 784

In [12]:
img = torch.randn(64, 28*28)

- checking image size

In [13]:
img.shape # 64 images, and 784 flatten image pixels

torch.Size([64, 784])

- initalized model, which takes input `784` and outputs `10` tensors

In [14]:
model = NN(28*28, 10)

- passing all 64 a one batch of image, and model should simultaneously generate the result as 64 rows and 10 columns corresponding scores w.r.t each class

In [16]:
model(img).shape

torch.Size([64, 10])

In [18]:
# model(img)

### Set Device

In [19]:
device = "cpu"

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

In [21]:
device = torch.device(device)
device

device(type='mps')

### Hyperparameters

In [47]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

### Load Data

In [48]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loder = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loder = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Initalize Networks

In [49]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

### Loss and Optimizers

In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [55]:
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loder):
        # get data to MPS if possible
        data = data.to(device)
        target = target.to(device)
        # getting it into correct shape
        # unrolling the image
        data = data.reshape(data.shape[0], -1) # 1st dimension will remain as it, rest will be flattened

        # forward pass
        scores = model(data)
        loss = criterion(scores, target) # prediction , ground truth

        # backward
        optimizer.zero_grad() # setting gradients to zero for each batch
        loss.backward()

        # gradient descent or Adam step
        optimizer.step()
    print(f"Epoch {epoch}: Loss: {loss}")

Epoch 0: Loss: 0.33647382259368896
Epoch 1: Loss: 0.05019011348485947
Epoch 2: Loss: 0.07575917989015579
Epoch 3: Loss: 0.06630337983369827
Epoch 4: Loss: 0.016096752136945724


### Check accuracy on train & test to see how good our model

In [56]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on testing data")
    num_correct = 0
    num_samples = 0
    model.eval() # puts model into evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            x = x.reshape(x.shape[0], -1)

            scores = model(x)

            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()

            num_samples += predictions.size(0)

        acc = float(num_correct)/float(num_samples) * 100

        print(f"Got {num_correct} / {num_samples} with accuracy {acc:.2f}") 
    model.train()

    # return acc

In [57]:
check_accuracy(train_loder, model)

Checking accuracy on training data
Got 59096 / 60000 with accuracy 98.49


In [58]:
check_accuracy(test_loder, model)

Checking accuracy on training data
Got 59096 / 60000 with accuracy 98.49
